In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, recall_score,precision_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE,RandomOverSampler
from sklearn.ensemble import AdaBoostRegressor,AdaBoostClassifier
import xgboost as xgb
from sklearn.model_selection import cross_validate,GridSearchCV,RandomizedSearchCV
import joblib
import warnings            
warnings.filterwarnings("ignore") 

In [2]:
# loading the dataset.
df = pd.read_csv('../data/processed/processed_data_with_outliers.csv')
df

,age_group,length_of_stay,type_of_admission,ccs_diagnosis_description,ccs_procedure_description,apr_drg_description,apr_mdc_description,apr_severity_of_illness_description,apr_risk_of_mortality,gender_M,apr_medical_surgical_description_Surgical,emergency_department_indicator_Y
0,3,5,8.63,16.72,7.57,19.22,18.55,1,1,0,0,1
1,1,8,8.63,16.72,7.57,19.22,18.55,2,1,0,0,1
2,2,8,8.63,16.72,7.57,19.22,18.55,1,1,1,0,1
3,3,4,8.63,6.22,7.57,4.97,8.57,1,1,0,0,1
4,3,3,8.63,6.89,7.57,6.20,8.61,2,2,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
722448,3,3,8.63,9.32,6.79,5.76,7.81,3,3,0,1,1
722449,1,1,8.63,4.97,8.38,5.04,5.84,3,3,0,0,1
722450,4,3,8.63,7.68,4.88,7.40,7.81,3,3,1,0,1
722451,4,2,8.63,14.06,6.65,17.10,12.42,4,4,0,1,1


## 1) Maintain the huge size of the data set which will be challenging when appling diffrent modeling approch:
#### Although the size of the data after cleaning and preprocessing decreese greatly to 722,453 entries instance and 12 features but it is still challenging to deal with it for limted pressoer laptob so we are going take small sample to run the models more efficently while maintaing the same characteristics of the data by usin straitify method.

In [3]:
df.length_of_stay = df.length_of_stay.astype(float)

In [4]:
df.length_of_stay

0         5.0
1         8.0
2         8.0
3         4.0
4         3.0
         ... 
722448    3.0
722449    1.0
722450    3.0
722451    2.0
722452    2.0
Name: length_of_stay, Length: 722453, dtype: float64

In [5]:
# divede the data fram to x = features & y = target
x = df.drop("length_of_stay", axis = 1)
y = df["length_of_stay"]

In [6]:
# taking sample of the data using stratify to maintain data characteristics
main_x,sample_x,main_y,sample_y = train_test_split(x,y,test_size=0.01,stratify=y)

In [7]:
sample_y.value_counts()

2.0     848
3.0     828
1.0     758
4.0     713
5.0     593
       ... 
73.0      1
84.0      1
72.0      1
91.0      1
95.0      1
Name: length_of_stay, Length: 99, dtype: int64

In [8]:
sampler = RandomOverSampler()
sample_x,sample_y = sampler.fit_resample(sample_x,sample_y)

In [9]:
sample_x.head()

,age_group,type_of_admission,ccs_diagnosis_description,ccs_procedure_description,apr_drg_description,apr_mdc_description,apr_severity_of_illness_description,apr_risk_of_mortality,gender_M,apr_medical_surgical_description_Surgical,emergency_department_indicator_Y
0,1,8.63,3.83,7.57,3.75,8.61,1,1,0,0,1
1,1,8.63,6.70,7.49,5.12,7.33,3,2,0,0,1
2,4,8.63,7.44,5.14,6.04,7.64,2,2,1,0,1
3,4,8.63,4.64,10.40,4.23,5.34,3,2,0,0,1
4,2,8.63,6.96,7.66,7.11,7.33,2,2,1,1,0


In [10]:
sample_y.value_counts()

1.0      848
75.0     848
57.0     848
84.0     848
53.0     848
        ... 
120.0    848
39.0     848
37.0     848
36.0     848
95.0     848
Name: length_of_stay, Length: 99, dtype: int64

In [11]:
sample_y.count()

83952

In [12]:
sample_y.isnull().sum()

0

In [13]:
sample_x.isnull().sum()

age_group                                    0
type_of_admission                            0
ccs_diagnosis_description                    0
ccs_procedure_description                    0
apr_drg_description                          0
apr_mdc_description                          0
apr_severity_of_illness_description          0
apr_risk_of_mortality                        0
gender_M                                     0
apr_medical_surgical_description_Surgical    0
emergency_department_indicator_Y             0
dtype: int64

In [14]:
x_train,x_test,y_train,y_test = train_test_split(sample_x,sample_y,test_size=0.5,stratify=sample_y)

In [15]:
# Using Stander Scaler for x_train, x_test
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [16]:
# trying three different models to compare between them
models = {"lr": LinearRegression(),
          "knn": KNeighborsRegressor(),
          "rf": RandomForestRegressor(),
         "Adboost": AdaBoostRegressor()}

In [17]:
# running the three different models and check the result using for loop
for name,model in models.items():
    print("----------- ", name, " -------------")
    model.fit(x_train_scaled,y_train)
    y_train_pred = model.predict(x_train_scaled)
    y_test_pred = model.predict(x_test_scaled)
    print("Train Acurracy Score", round(model.score(x_train_scaled,y_train),2),"Test Acurracy Score", round(model.score(x_test_scaled,y_test),2))
    print("Train MAE Score: ", mean_absolute_error(y_train_pred,y_train),"Test MAE Score: ", round(mean_absolute_error(y_test_pred,y_test),2))
    print("*"*60)

-----------  lr  -------------
Train Acurracy Score 0.3 Test Acurracy Score 0.29
Train MAE Score:  19.709664741608545 Test MAE Score:  19.79
************************************************************
-----------  knn  -------------
Train Acurracy Score 0.98 Test Acurracy Score 0.98
Train MAE Score:  0.7617543358109397 Test MAE Score:  1.0
************************************************************
-----------  rf  -------------
Train Acurracy Score 0.99 Test Acurracy Score 0.99
Train MAE Score:  0.48379897572297653 Test MAE Score:  0.75
************************************************************
-----------  Adboost  -------------
Train Acurracy Score 0.33 Test Acurracy Score 0.33
Train MAE Score:  20.32362627267084 Test MAE Score:  20.32
************************************************************


In [18]:
knnr = KNeighborsRegressor()

In [19]:
knnr.fit(x_train_scaled,y_train)
y_train_pred = knnr.predict(x_train_scaled)
y_test_pred = knnr.predict(x_test_scaled)
print("Train Acurracy Score", round(knnr.score(x_train_scaled,y_train),2),"Test Acurracy Score", round(knnr.score(x_test_scaled,y_test),2))
print("Train MAE Score: ", mean_absolute_error(y_train_pred,y_train),"Test MAE Score: ", round(mean_absolute_error(y_test_pred,y_test),2))
print("*"*60)

Train Acurracy Score 0.98 Test Acurracy Score 0.98
Train MAE Score:  0.7617543358109397 Test MAE Score:  1.0
************************************************************


In [20]:
rfr = RandomForestRegressor()

In [21]:
rfr.fit(x_train_scaled,y_train)
y_train_pred = rfr.predict(x_train_scaled)
y_test_pred = rfr.predict(x_test_scaled)
print("Train Acurracy Score", round(rfr.score(x_train_scaled,y_train),2),"Test Acurracy Score", round(rfr.score(x_test_scaled,y_test),2))
print("Train MAE Score: ", mean_absolute_error(y_train_pred,y_train),"Test MAE Score: ", round(mean_absolute_error(y_test_pred,y_test),2))
print("*"*60)

Train Acurracy Score 0.99 Test Acurracy Score 0.99
Train MAE Score:  0.4873045281968156 Test MAE Score:  0.75
************************************************************


## Parameter Tuning

#### Here are some parameters that we can use for a random forest regressor model for cross-validation with grid & randomize search:

        * n_estimators: The number of trees in the forest.
        * max_depth: The maximum depth of each tree in the forest.
        * min_samples_split: The minimum number of samples required to split a node in the tree.
        * min_samples_leaf: The minimum number of samples required in a leaf node.
        * max_features: The maximum number of features to consider when splitting a node in the tree.
        * bootstrap: Whether to bootstrap the data when training the trees in the forest.
        * criterion: The criterion used to split nodes in the trees.
        * oob_score: Whether to calculate the out-of-bag (OOB) score during training.
        * cv: The number of folds to use in cross-validation.
        * n_iter: The number of iterations to perform for randomized search.

In [22]:
# Define parameter grid
param_grid = {
    'n_estimators': [50,100,150,200],
    'max_depth': [2,5,10],
    'min_samples_split': [2,5,10],
    'min_samples_leaf': [2,5,10],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False],
    'oob_score': [True, False]
}

# Perform grid search
random_search = RandomizedSearchCV(rfr, param_grid, cv=5, n_iter=100)
random_search.fit(x_train_scaled,y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [2, 5, 10],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [2, 5, 10],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 150, 200],
                                        'oob_score': [True, False]})

In [23]:
# Get the best parameters
best_params = random_search.best_params_
best_params

{'oob_score': False,
 'n_estimators': 200,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': False}

In [24]:
random_search.best_estimator_

RandomForestRegressor(bootstrap=False, max_depth=10, max_features='sqrt',
                      min_samples_leaf=2, min_samples_split=10,
                      n_estimators=200)

In [25]:
y_train_pred = random_search.best_estimator_.predict(x_train_scaled)
y_test_pred = random_search.best_estimator_.predict(x_test_scaled)
print("Train Acurracy Score", round(rfr.score(x_train_scaled,y_train),2),"Test Acurracy Score", round(rfr.score(x_test_scaled,y_test),2))
print("Train MAE Score: ", mean_absolute_error(y_train_pred,y_train),"Test MAE Score: ", round(mean_absolute_error(y_test_pred,y_test),2))
print("*"*60)

Train Acurracy Score 0.99 Test Acurracy Score 0.99
Train MAE Score:  5.873065263906387 Test MAE Score:  5.97
************************************************************


In [26]:
# list for features names
features = list(x_train.columns)
features

['age_group',
 'type_of_admission',
 'ccs_diagnosis_description',
 'ccs_procedure_description',
 'apr_drg_description',
 'apr_mdc_description',
 'apr_severity_of_illness_description',
 'apr_risk_of_mortality',
 'gender_M',
 'apr_medical_surgical_description_Surgical',
 'emergency_department_indicator_Y']

In [27]:
# saving random search bet model estimator to variables
rfr_model = random_search.best_estimator_

In [28]:
# creating h5 files for randomforest models for length of stay
joblib.dump(knnr,"../models/knnr.h5")
joblib.dump(rfr,"../models/rfr.h5")
joblib.dump(rfr_model,"../models/rfr_model.h5")
joblib.dump(features,"../models/features.h5")
joblib.dump(scaler,"../models/scaler.h5")

['../models/scaler.h5']

# **Summary:**

Although the model demonstrated an impressive training and testing accuracy of 99%, the results were based on a minimal 0.01% sample size of the original data, carefully selected using stratified sampling. While this model shows promise, it cannot yet be considered ready for generalization. To achieve that level of trust and applicability, further investment in terms of time, computational resources, and extensive studies is required.

Throughout this project, numerous techniques were explored in several different notebooks of preprocessing & modeling, encompassing feature selection, outlier handling, diverse encoding methods for various classification and regression models, and cross-validation techniques. These endeavors represent a comprehensive effort to develop a robust predictive model for inpatient length of stay. 

The journey has been informative, revealing both the potential of machine learning in healthcare and the challenges associated with working on real-world, large-scale datasets. Continued research, collaboration, and refinements are essential for building a truly reliable and widely applicable solution.
